# Inżynieria lingwistyczna
Ten notebook jest oceniany półautomatycznie. Nie twórz ani nie usuwaj komórek - struktura notebooka musi zostać zachowana. Odpowiedź wypełnij tam gdzie jest na to wskazane miejsce - odpowiedzi w innych miejscach nie będą sprawdzane (nie są widoczne dla sprawdzającego w systemie).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE".

---

# Moduł 5: Statystyczne tłumaczenie maszynowe

## Zadanie 1
Zadanie polega na zaimplementowaniu algorytmu Expectation-Maximization w modelu IBM Model 1 do przypasowywania słów. Będzie to fragment modelu, który tłumaczyć będzie z hiszpańskiego na angielski. 

UWAGA: Specjalny token "NULL" pomijamy w implementacji.

Dany jest mini-korpus równoległy angielsko-hiszpański
- "green house" "casa verde"
- "the house" "la casa"
- "the green house" "la casa verde"


In [0]:
import itertools
english = [["green","house"], ["the","house"], ["the", "green", "house"]]
spanish = [["casa", "verde"], ["la", "casa"], ["la", "casa", "verde"]]

W dalszych funkcjach przydatne może być wyznaczenie słownika czyli zbioru słów z korpusu dla danego języka.

In [0]:
def get_vocabulary(corpus):
    """
    Funkcja zwracająca zbiór unikalnych słów z korpusu podanego w formacie zmiennej english i spanish
    """
    return set(word for nested_list in corpus for word in nested_list)

In [0]:
from nose.tools import assert_set_equal
assert_set_equal(set(get_vocabulary(english)), set(["the", "green", "house"]))

Zainicjalizuj rozkład prawdopodobieństwa tłumaczenia słów rozkładem jednorodnym. Ponieważ zależy nam na prostocie implementacji (a nie efektywności) możemy to prawdopodobieństwo zaimplementować jako zwykły słownik, który będzie przyjmował na wejściu krotkę dwóch słów. Słownik nazwij `translation_prob` z kluczami (słowo_es, słowo_en).

In [0]:
from collections import defaultdict

def initalize_translation_prob(target, source):
  """
  Inicjalizacja prawdopodobieństw tłumaczeń pomiędzy słowami.
  Wyjściowy słownik w postaci (słowo z source, słowo z target) => prawdopodobieństwo

  Początkowe prawdopodobieństwa jako rozkład jednorodny.
  """
  prob = {}
  words_in_target = get_vocabulary(target)
  words_in_source = get_vocabulary(source)
  uniform_probability = 1 / len(words_in_target)

  for word_target in words_in_target:
    for word_source in words_in_source:
      prob[(word_source, word_target)] = uniform_probability
  return prob

translation_prob = initalize_translation_prob(english, spanish)

Wypisz zaincjalizowany słownik, żeby upewnić się że wynik jest prawidłowy.

In [5]:
translation_prob

{('casa', 'green'): 0.3333333333333333,
 ('casa', 'house'): 0.3333333333333333,
 ('casa', 'the'): 0.3333333333333333,
 ('la', 'green'): 0.3333333333333333,
 ('la', 'house'): 0.3333333333333333,
 ('la', 'the'): 0.3333333333333333,
 ('verde', 'green'): 0.3333333333333333,
 ('verde', 'house'): 0.3333333333333333,
 ('verde', 'the'): 0.3333333333333333}

Zaimplementuj pierwszy krok algorytmu EM. Wyznacz wartości oczekiwane zmiennych przypisania słowa we wszystkich zdaniach w korpusie (oznaczane na wykładzie jako `a`).

In [0]:
from collections import defaultdict

def calculate_expectation(corpora1, corpora2, translation_prob):
    """
    Procedura wykonująca krok "E" algorytmu EM
    Wynikiem wartości oczekiwane dla zmiennej przypisań słów w zdaniach w postaci
    słownika (słowo z języka źródłowego, słowo z języka docelowego) => wartość.
    """
    source_target = {}

    for sentence_k_number, (target_sentence, source_sentence) in enumerate(zip(corpora1, corpora2)):
      for target in target_sentence:
        for source in source_sentence:
          source_target[(sentence_k_number, source, target)] = translation_prob.get((source, target), 0)

    # Normalizacja
    totals_probs = defaultdict(lambda: 0.0)
    for (iteration, source, target), value in source_target.items():
        totals_probs[(iteration, target)] += value
    return { (iteration, source, target): value / totals_probs[(iteration, target)] for (iteration, source, target), value in source_target.items()}

assigment_expected_values = calculate_expectation(english, spanish, translation_prob)

Wypisz wartości oczekiwane zmiennych przypisań, aby zobaczyć jak wyglądają. Powinny one również prezentować całkowity brak wiedzy o przypisaniach (rozkłady jednorodne).

In [7]:
assigment_expected_values

{(0, 'casa', 'green'): 0.5,
 (0, 'casa', 'house'): 0.5,
 (0, 'verde', 'green'): 0.5,
 (0, 'verde', 'house'): 0.5,
 (1, 'casa', 'house'): 0.5,
 (1, 'casa', 'the'): 0.5,
 (1, 'la', 'house'): 0.5,
 (1, 'la', 'the'): 0.5,
 (2, 'casa', 'green'): 0.3333333333333333,
 (2, 'casa', 'house'): 0.3333333333333333,
 (2, 'casa', 'the'): 0.3333333333333333,
 (2, 'la', 'green'): 0.3333333333333333,
 (2, 'la', 'house'): 0.3333333333333333,
 (2, 'la', 'the'): 0.3333333333333333,
 (2, 'verde', 'green'): 0.3333333333333333,
 (2, 'verde', 'house'): 0.3333333333333333,
 (2, 'verde', 'the'): 0.3333333333333333}

Zaimplementuj drugi krok algorytmu EM. Wyznacz nowe `translation_prob` na podstawie oczekiwanych wartości zmiennych przypisań.

In [0]:
def calculate_maximization(corpora1, corpora2, assigment_expected_values):
  """
  Procedura wykonująca krok "M" algorytmu EM

  Normalizacja prawdopodobieństw warunkowych dla słów.
  """
  t_source_target = defaultdict(lambda: 0.0)
  total_word_prob = defaultdict(lambda: 0.0)

  for (sentence_k_number, source, target), prob in assigment_expected_values.items():
    t_source_target[(source, target)] += prob
    total_word_prob[target] += prob

  # Normalizacja
  return { (source, target): value / total_word_prob[target] for (source, target), value in t_source_target.items()}

translation_prob = calculate_maximization(english, spanish, assigment_expected_values)

In [0]:
from nose.tools import assert_almost_equal
assert_almost_equal(translation_prob[('casa', 'house')], 4/9.)
assert_almost_equal(translation_prob[('la', 'house')], 5/18.)

Wywołaj w pętli 10 kroków algorytmu EM i zaobserwuj jak zmieniają się prawdopodobieństwa dla tłumacznienia "house".

In [10]:
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish, translation_prob)
    translation_prob = calculate_maximization(english, spanish, assigment_expected_values)
    print([(i,j) for i,j in translation_prob.items() if i[1] == "house"])
    print("---")


[(('casa', 'house'), 0.5584045584045584), (('verde', 'house'), 0.22079772079772084), (('la', 'house'), 0.22079772079772084)]
---
[(('casa', 'house'), 0.6638923177619094), (('verde', 'house'), 0.16805384111904528), (('la', 'house'), 0.16805384111904528)]
---
[(('casa', 'house'), 0.7532968646774504), (('verde', 'house'), 0.12335156766127481), (('la', 'house'), 0.12335156766127481)]
---
[(('casa', 'house'), 0.8239601969358895), (('verde', 'house'), 0.08801990153205524), (('la', 'house'), 0.08801990153205524)]
---
[(('casa', 'house'), 0.8769766282684489), (('verde', 'house'), 0.061511685865775545), (('la', 'house'), 0.061511685865775545)]
---
[(('casa', 'house'), 0.915296630096382), (('verde', 'house'), 0.042351684951809056), (('la', 'house'), 0.042351684951809056)]
---
[(('casa', 'house'), 0.94228242707854), (('verde', 'house'), 0.028858786460729976), (('la', 'house'), 0.028858786460729976)]
---
[(('casa', 'house'), 0.9609498992371662), (('verde', 'house'), 0.019525050381416956), (('la', 

Wywołaj algorytm EM na poniższym korpusie.

In [0]:
english2 = [["the","dog"], ["the","house"], ["the", "green", "house"]]
polish = [["pies"], ["dom"], ["zielony", "dom"]]

In [0]:
translation_prob = initalize_translation_prob(english2, polish)
for i in range(10):
    assigment_expected_values = calculate_expectation(english2, polish, translation_prob)
    translation_prob = calculate_maximization(english2, polish, assigment_expected_values)

Sprawdź jak wyglądają prawdopodobieństwa tłumaczeń po 10 iteracjach.

In [13]:
translation_prob

{('dom', 'green'): 0.5,
 ('dom', 'house'): 0.99951171875,
 ('dom', 'the'): 0.6663411458333334,
 ('pies', 'dog'): 1.0,
 ('pies', 'the'): 0.3333333333333333,
 ('zielony', 'green'): 0.5,
 ('zielony', 'house'): 0.00048828125,
 ('zielony', 'the'): 0.0003255208333333333}

Sprawdź czy gdybyś dodał słówko `NULL` to algorytm nauczyłby się wiązać słówko `NULL` na `the`, które nie występuje w języku polskim?

In [14]:
polish_with_null = [["NULL"] + sentence for sentence in polish]
polish_with_null

[['NULL', 'pies'], ['NULL', 'dom'], ['NULL', 'zielony', 'dom']]

In [15]:
# Analiza english & polish z NULL
translation_prob = initalize_translation_prob(english2, polish_with_null)
for i in range(10):
    assigment_expected_values = calculate_expectation(english2, polish_with_null, translation_prob)
    translation_prob = calculate_maximization(english2, polish_with_null, assigment_expected_values)

for (k, v) in translation_prob.items():
  if v > 0.01:
    print('{:10} {:10} {}'.format(k[0], k[1], v))

# --------------------------------------------------------------------------------------
print('-------------------------------------------------------------------------------')
# --------------------------------------------------------------------------------------

# Analiza english & spanish z NULL
spanish_with_null = [["NULL"] + sentence for sentence in spanish]
translation_prob = initalize_translation_prob(english, spanish_with_null)
for i in range(10):
    assigment_expected_values = calculate_expectation(english, spanish_with_null, translation_prob)
    translation_prob = calculate_maximization(english, spanish_with_null, assigment_expected_values)

for (k, v) in translation_prob.items():
  if v > 0.01:
    print('{:10} {:10} {}'.format(k[0], k[1], v))

NULL       the        0.9884935162981091
NULL       dog        0.5
pies       dog        0.5
dom        the        0.01146338484277804
NULL       house      0.49983723958333337
dom        house      0.49983723958333337
NULL       green      0.3333333333333333
zielony    green      0.3333333333333333
dom        green      0.3333333333333333
-------------------------------------------------------------------------------
NULL       green      0.333251953125
casa       green      0.333251953125
verde      green      0.333251953125
NULL       house      0.4927324333313708
casa       house      0.4927324333313708
NULL       the        0.33325195312499994
la         the        0.33325195312499994
casa       the        0.33325195312499994


Jeśli wywołałbyś EM dla pierwszego korpusu równoległego (zmienne `english` i `spanish`) i dołączył tokeny `NULL` to EM tłumaczy NULL jako "casa" i "house" jako "casa" z takimi samymi prawdopodobieństwami. Dlaczego?

Może to wynikać z założeń i ograniczeń które czyni model IBM 1:
* słowo emitowane jest z dokładnie jednego słowa z języka źródłowego - nasz model chce się nauczyć że `NULL` też ma zastosowanie (co w przypadku danych w korpusie ES <> EN nie ma miejsca - każde słowo ma swój odpowiednik).